In [ ]:
!pip install transformers tensorflow numpy scikit-learn

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Method/EnglishMethod/Dataset2.csv') # English or Chinese Dataset
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)
# Initialize an empty list to store similarity scores
# Define the number of top rows to calculate similarity for (e.g., 3000)
top_rows = 3000

# Initialize an empty list to store similarity scores
similarity_scores = []

# Iterate through the first 'top_rows' rows and calculate similarity scores
for index, row in df.head(top_rows).iterrows():
    # Tokenize and encode the sentences
    encoded_descriptions = tokenizer(row["Functional Description"], padding=True, truncation=True, return_tensors="tf")
    encoded_method_names = tokenizer(row["Method Name"], padding=True, truncation=True, return_tensors="tf")

    # Generate embeddings for the sentences
    embed_descriptions = model(encoded_descriptions).last_hidden_state[:, 0, :].numpy()
    embed_method_names = model(encoded_method_names).last_hidden_state[:, 0, :].numpy()

    # Calculate cosine similarity between embeddings
    similarity_score = cosine_similarity(embed_descriptions, embed_method_names)[0][0]

    # Append the similarity score to the list
    similarity_scores.append(similarity_score)

# Add similarity scores to the first 'top_rows' rows of the DataFrame
df.loc[:top_rows - 1, "similarity_score"] = similarity_scores

# Print the DataFrame with similarity scores
print(df)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)ultilingual-cased/resolve/main/vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

(…)ingual-cased/resolve/main/tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

(…)tilingual-cased/resolve/main/config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

                                  Functional Description  \
0                         get the role of this object      
1                        get the state of this object      
2      called by the browser or applet viewer to info...   
3      gets the accessible context associated with th...   
4      determines this applet ' s context , which all...   
...                                                  ...   
17734  allows an interceptor to query the information...   
17735  this operation is invoked when the orb receive...   
17736  returns the object id identifying the object i...   
17737  returns reference to the poa implementing the ...   
17738  receives requests issued to any corba object i...   

                 Method Name  similarity_score  
0          getAccessibleRole          0.813586  
1      getAccessibleStateSet          0.751116  
2                    destroy          0.343025  
3       getAccessibleContext          0.753515  
4           getAppletContext      

In [ ]:
# Define the scoring function
def calculate_method_name_score(method_name):
    score = 0

    # Rule 1: Use of Verb-Noun Pairs
    if any(word.isalpha() and word[0].islower() for word in method_name.split()):
        score += 0.2

    # Rule 2: CamelCase Formatting
    if method_name.isidentifier():
        score += 0.2

    # Rule 3: Specificity
    if len(method_name) <= 20:  # Adjust the length threshold as needed
        score += 0.2

    # Rule 4: Avoiding Java Reserved Words (you can define a list of reserved words to check against)
    reserved_words = ["public", "private", "static", "int", "abstract", "assert", "boolean", "break", "byte", "case",
                      "catch", "char", "class", "const", "continue", "default", "do", "double", "else", "enum",
                      "extends", "final", "finally", "float", "for", "goto", "if", "implements", "import",
                      "instanceof", "interface", "long", "native", "new", "package", "protected", "public",
                      "return", "short", "static", "strictfp", "super", "switch", "synchronized", "this", "throw",
                      "throws", "transient", "try", "void", "volatile", "while"]  # Define the list
    if method_name not in reserved_words:
        score += 0.2

    # Rule 5: Avoiding Special Characters and Spaces
    if all(c.isalnum() or c == '_' for c in method_name):
        score += 0.2

    return score

# Add the "EvlScore" column to the DataFrame
df["EvlScore"] = df["Method Name"].apply(calculate_method_name_score)

# Print the DataFrame with the "EvlScore" column
print(df)


                                  Functional Description  \
0                         get the role of this object      
1                        get the state of this object      
2      called by the browser or applet viewer to info...   
3      gets the accessible context associated with th...   
4      determines this applet ' s context , which all...   
...                                                  ...   
17734  allows an interceptor to query the information...   
17735  this operation is invoked when the orb receive...   
17736  returns the object id identifying the object i...   
17737  returns reference to the poa implementing the ...   
17738  receives requests issued to any corba object i...   

                 Method Name  similarity_score  EvlScore  
0          getAccessibleRole          0.813586       1.0  
1      getAccessibleStateSet          0.751116       0.8  
2                    destroy          0.343025       1.0  
3       getAccessibleContext          0.753

In [ ]:
# Get the first 3000 rows and assign them to a new DataFrame
df_first_3000 = df.head(3000)

# Save the new DataFrame to a CSV file
df_first_3000.to_csv('/content/drive/MyDrive/Method/EnglishMethod/Dataset3.csv', index=False)

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Method/EnglishMethod/Dataset3.csv')

# Sort the DataFrame by similarity_score and EvlScore in descending order
df_sorted = df.sort_values(by=["similarity_score", "EvlScore"], ascending=[False, False])

# Get the top 1000 pairs
QExpDS = df_sorted.head(1800)
QExpDS.to_csv('/content/drive/MyDrive/Method/EnglishMethod/QExDS.csv', index=False)
